In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

df_2010_2011 = pd.read_excel("online_retail.xlsx", sheet_name = "Year 2010-2011")
df = df_2010_2011.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
Invoice        541910 non-null object
StockCode      541910 non-null object
Description    540456 non-null object
Quantity       541910 non-null int64
InvoiceDate    541910 non-null datetime64[ns]
Price          541910 non-null float64
Customer ID    406830 non-null float64
Country        541910 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
# Veri setimizde eksik veriler bulunmakta.
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [4]:
# Eksik verileri doğrudan veri setinden kaldırıyoruz.
df.dropna(inplace = True) 

In [5]:
# Veri setimizde kaç farklı ürün olduğunu görmek için 
df["Description"].nunique()

3896

In [7]:
# En fazla satılan ilk 5 ürünlerimize bir göz atmak istedim.
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity",ascending= False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


In [8]:
# Siparişlerden elde edilen toplam kazancımızı görmek için yeni bir değişken atamamız gerekiyor.
df["Total_Price"] = df["Quantity"] * df["Price"]

In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [12]:
# En fazla gelir elde ettğimiz siparişlerimizi görüyoruz.
df.groupby("Invoice").agg({"Total_Price":"sum"}).sort_values("Total_Price",ascending = False).head()

,Total_Price
Invoice,
581483,168469.60
541431,77183.60
556444,38970.00
567423,31698.16
556917,22775.93


In [13]:
# Veri setimizde iptal edilen siparişlerde bulunmakta bunları önce gözlemleyip sonrasında veri setimizden kaldıralım.
df[df["Invoice"].str.contains("C",na = False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96


In [16]:
df1 = df_2010_2011.copy()

In [20]:
# İade edilen ürünlerimizden yeni bir df yaratıp sonrasında en fazla iade alan ürünlerimizi görmek istiyorum.
df2 = df1[df1["Invoice"].str.contains("C",na = False)]

In [21]:
# En fazla iade olan 5 ürün
df2["Description"].value_counts().head()

Manual                      244
REGENCY CAKESTAND 3 TIER    181
POSTAGE                     126
JAM MAKING SET WITH JARS     87
Discount                     77
Name: Description, dtype: int64

In [22]:
# İadeli siparişlerimizi veri setimizden kaldırıyoruz.
df = df[~df["Invoice"].str.contains("C",na = False)]

In [23]:
# Ülke bazında kazançlarımızı ve satışlarımızı görelim
df.groupby("Country").agg({"Quantity":"sum",
                           "Total_Price":"sum"}).sort_values("Quantity",ascending = False).head()

,Quantity,Total_Price
Country,,
United Kingdom,4269472,7.308392e+06
Netherlands,200937,2.854463e+05
EIRE,140525,2.655459e+05
Germany,119263,2.288671e+05
France,111473,2.090420e+05


In [26]:
# Veri setimizin istatistiklerine bir göz gezdirmek istedim. 
#İstatistiklere kabaca bakacak olursak aykırı değerler olduğu gözlemlenmekte bu çalışmada aykırı değerler için herhangi bir işlemde bulunmayacağım
df.describe([0.01,0.25,0.50,0.75,0.90,0.99]).T

,count,mean,std,min,1%,25%,50%,75%,90%,99%,max
Quantity,397925.0,13.021793,180.419984,1.0,1.00,2.00,6.00,12.00,24.00,120.00,80995.00
Price,397925.0,3.116212,22.096773,0.0,0.21,1.25,1.95,3.75,6.35,14.95,8142.75
Customer ID,397925.0,15294.308601,1713.172738,12346.0,12415.00,13969.00,15159.00,16795.00,17725.00,18211.00,18287.00
Total_Price,397925.0,22.394737,309.055200,0.0,0.55,4.68,11.80,19.80,35.40,202.50,168469.60


## RFM ANALİZİ İÇİN METRİKLERİN HESAPLANMASI

### Recency:

In [27]:
# Veri setimizi tekrardan bir görelim
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [28]:
# Recency değerimizi hesaplamak için öncelikle güncel bir tarih oluşturuyoruz. 
# Tarihi oluştururken en yakın geliş tarihinin bir gün sonrasını aldık yoksa bazı hesaplamalarda recency değerimiz -1 çıkabiliyor.
import datetime as dt
today_date = dt.datetime(2011,12,10)

In [ ]:
# Customer ID'deki ondalıklı görüntüden kurtulmak istedim.
df["Customer ID"] = df["Customer ID"].astype(int)

In [29]:
# Recency değerimizi yaratmak için gerekli hesaplamayı yaptırıyoruz.
recency = today_date - (df.groupby("Customer ID").agg({"InvoiceDate":"max"}))

In [30]:
recency.head()

,InvoiceDate
Customer ID,
12346.0,325 days 13:59:00
12347.0,2 days 08:08:00
12348.0,75 days 10:47:00
12349.0,18 days 14:09:00
12350.0,310 days 07:59:00


In [31]:
# InvoiceData sütun adından kurtulmak istedim.
recency.rename(columns = {"InvoiceDate":"Recency"},inplace = True)

In [32]:
recency.head()

,Recency
Customer ID,
12346.0,325 days 13:59:00
12347.0,2 days 08:08:00
12348.0,75 days 10:47:00
12349.0,18 days 14:09:00
12350.0,310 days 07:59:00


In [33]:
# Recency değerlerimizdeki bozukluk devam ediyor gün ve saattende kurtulmak istiyorum:
recency_df = recency["Recency"].apply(lambda x: x.days)

In [34]:
recency_df.head()

Customer ID
12346.0    325
12347.0      2
12348.0     75
12349.0     18
12350.0    310
Name: Recency, dtype: int64

### Frequency

In [35]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [36]:
# Müşterilerimizin alışveriş sayılarını görmek istedim.
df.groupby("Customer ID").agg({"Invoice": "nunique"}).sort_values("Invoice",ascending = False).head()

,Invoice
Customer ID,
12748.0,210
14911.0,201
17841.0,124
13089.0,97
14606.0,93


In [37]:
df_freq = df.groupby("Customer ID").agg({"Invoice": "nunique"})

In [38]:
df_freq.head()

,Invoice
Customer ID,
12346.0,1
12347.0,7
12348.0,4
12349.0,1
12350.0,1


In [39]:
df_freq.rename(columns = {"Invoice":"Frequincy"},inplace = True) 

In [40]:
df_freq.head()

,Frequincy
Customer ID,
12346.0,1
12347.0,7
12348.0,4
12349.0,1
12350.0,1


### Monetary

In [41]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [44]:
df.groupby("Customer ID").agg({"Total_Price":"sum"}).sort_values("Total_Price",ascending = False).head()

,Total_Price
Customer ID,
14646.0,280206.02
18102.0,259657.30
17450.0,194550.79
16446.0,168472.50
14911.0,143825.06


In [45]:
monetary_df = df.groupby("Customer ID").agg({"Total_Price":"sum"})

In [46]:
monetary_df.rename(columns = {"Total_Price":"Monetary"},inplace = True)

In [47]:
monetary_df.head()

,Monetary
Customer ID,
12346.0,77183.60
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40


### RFM DF Oluşturma

In [48]:
rfm = pd.concat([recency_df,df_freq,monetary_df],axis = 1)

In [49]:
rfm.head()

,Recency,Frequincy,Monetary
Customer ID,,,
12346.0,325,1,77183.60
12347.0,2,7,4310.00
12348.0,75,4,1797.24
12349.0,18,1,1757.55
12350.0,310,1,334.40


### Skorların Oluşturulması

In [51]:
rfm["RecencyScore"] = pd.qcut(rfm["Recency"],5,labels = [5,4,3,2,1])

In [52]:
rfm.head()

,Recency,Frequincy,Monetary,RecencyScore
Customer ID,,,,
12346.0,325,1,77183.60,1
12347.0,2,7,4310.00,5
12348.0,75,4,1797.24,2
12349.0,18,1,1757.55,4
12350.0,310,1,334.40,1


In [53]:
rfm["FreqScore"] = pd.qcut(rfm["Frequincy"].rank(method = "first"),5,labels = [1,2,3,4,5])

In [54]:
rfm.head()

,Recency,Frequincy,Monetary,RecencyScore,FreqScore
Customer ID,,,,,
12346.0,325,1,77183.60,1,1
12347.0,2,7,4310.00,5,5
12348.0,75,4,1797.24,2,4
12349.0,18,1,1757.55,4,1
12350.0,310,1,334.40,1,1


In [57]:
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"],5,labels = [1,2,3,4,5])

In [58]:
rfm.head()

,Recency,Frequincy,Monetary,RecencyScore,FreqScore,MonetaryScore
Customer ID,,,,,,
12346.0,325,1,77183.60,1,1,5
12347.0,2,7,4310.00,5,5,5
12348.0,75,4,1797.24,2,4,4
12349.0,18,1,1757.55,4,1,4
12350.0,310,1,334.40,1,1,2


In [59]:
rfm["RFMScore"] = (rfm["RecencyScore"].astype(str)+rfm["FreqScore"].astype(str)+rfm["MonetaryScore"].astype(str))

In [60]:
rfm.head()

,Recency,Frequincy,Monetary,RecencyScore,FreqScore,MonetaryScore,RFMScore
Customer ID,,,,,,,
12346.0,325,1,77183.60,1,1,5,115
12347.0,2,7,4310.00,5,5,5,555
12348.0,75,4,1797.24,2,4,4,244
12349.0,18,1,1757.55,4,1,4,414
12350.0,310,1,334.40,1,1,2,112


In [63]:
# Segment sütunumuzuda ekliyoruz
rfm["Segment"] = (rfm["RecencyScore"].astype(str) +rfm["FreqScore"].astype(str))

In [64]:
rfm.head()

,Recency,Frequincy,Monetary,RecencyScore,FreqScore,MonetaryScore,RFMScore,Segment
Customer ID,,,,,,,,
12346.0,325,1,77183.60,1,1,5,115,11
12347.0,2,7,4310.00,5,5,5,555,55
12348.0,75,4,1797.24,2,4,4,244,24
12349.0,18,1,1757.55,4,1,4,414,41
12350.0,310,1,334.40,1,1,2,112,11


In [65]:
# RegEx kullanarak nihai olarak Segmentlerimizi oluşturmuş olacağız.
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [66]:
rfm["Segment"] = rfm["Segment"].replace(seg_map, regex = True)

In [67]:
rfm.head()

,Recency,Frequincy,Monetary,RecencyScore,FreqScore,MonetaryScore,RFMScore,Segment
Customer ID,,,,,,,,
12346.0,325,1,77183.60,1,1,5,115,Hibernating
12347.0,2,7,4310.00,5,5,5,555,Champions
12348.0,75,4,1797.24,2,4,4,244,At Risk
12349.0,18,1,1757.55,4,1,4,414,Promising
12350.0,310,1,334.40,1,1,2,112,Hibernating


In [68]:
rfm[["Segment","Recency","Frequincy", "Monetary"]].groupby("Segment").agg(["mean","count"])

Recency        Frequincy           Monetary      
                           mean count       mean count         mean count
Segment                                                                  
About to Sleep        52.312500   352   1.161932   352   471.994375   352
At Risk              152.785835   593   2.878583   593  1084.535297   593
Can't Loose          131.968254    63   8.380952    63  2796.155873    63
Champions              5.361769   633  12.417062   633  6857.963918   633
Hibernating          216.605042  1071   1.101774  1071   488.643307  1071
Loyal Customers       32.608059   819   6.479853   819  2864.247791   819
Need Attention        51.427807   187   2.326203   187   897.627861   187
New Customers          6.428571    42   1.000000    42   388.212857    42
Potential Loyalists   16.398760   484   2.010331   484  1041.222004   484
Promising             22.421053    95   1.000000    95   290.913158    95

## Yorumlama

### Champions

- Bu segmentte 633  müşterimiz bulunmakta  ortalama 5 gün önce gelmişler, aynı zamanda ortalama 12 defa alışveriş gerçekleştirmişler ve ortalama 6857 birimlik bir kazanç sağlanmış segmentimizden.
- "Champions" segmenti bizden çok sık alışveriş yapan ve güzel kazançlar getiren bir grup olduğu için çok fazla aksiyon gerektiren bir grup değil. Özel günlerini kutlamak , özel günlerinde onlara hediye çeki gibi göndermek gibi jestlerle onların mutlu olması sağlanabilir.

### Can't Loose

- Bu segmentte 63  müşterimiz bulunmakta, ortalama 131 gündür bir temasımız olmamış. Ancak Frequency ortalamasına bakıldığında ortalama 8 defa alışveriş gerçekleştirilmiş ve 2796 br kazanç elde edilmiş.
- "Can't Loose" segmenti son zamanlarda alışveriş yapmamış olsada geçmişte bizimle sık sık alışveriş yapmışlar. Böyle bir müşteri grubunu yeniden kazanmak isteriz. Onları yeniden kazanabilmek adına kendimizi hatırlatıcı mesajlar ve onları için yapılacak uygun indirimler ile yeniden bize gelmeleri sağlanabilir.

### New Customers

- Bu segmentte 42 müşterimizi bulunmakta ortalama 6 gün önce gelmişler. Yeni müşteri oldukları için frequency değerini 2den yüksek beklememek gerekir. Bize bu grubun kazancı ortalama 388 birim olmuş.
- "New Customers" segmenti potansiyel yeni müşterilerdir. Bize büyük kazançlarda getirebilirler. Bir daha gelmeyedebilirler. Biz tabii ki gelmelerini isteriz o yüzden çok fazla arayı açmadan onlara özel geliştirilecek indirimler ile onlara özel alışveriş teklifleriyle onlarla güçlü bir bağ yaratılabilir.